# Python + SQL Lab Exercise 9

Complete assignment with database creation, data insertion, and queries.

## 1. Database Setup

In [ ]:

import sqlite3

conn = sqlite3.connect("sales.db")
cur = conn.cursor()


## 2. Create Tables

In [ ]:

cur.executescript("""
DROP TABLE IF EXISTS CLIENT_MASTER;
DROP TABLE IF EXISTS PRODUCT_MASTER;
DROP TABLE IF EXISTS SALESMAN_MASTER;
DROP TABLE IF EXISTS SALES_ORDER;
DROP TABLE IF EXISTS SALES_ORDER_DETAILS;

CREATE TABLE CLIENT_MASTER (
    CLIENTNO TEXT PRIMARY KEY,
    NAME TEXT,
    CITY TEXT,
    PINCODE INTEGER,
    STATE TEXT,
    BALDUE REAL
);

CREATE TABLE PRODUCT_MASTER (
    PRODUCTNO TEXT PRIMARY KEY,
    DESCRIPTION TEXT,
    PROFITPERCENT REAL,
    UNITMEASURE TEXT,
    QTYONHAND INTEGER,
    REORDERLVL INTEGER,
    SELLPRICE REAL,
    COSTPRICE REAL
);

CREATE TABLE SALESMAN_MASTER (
    SALESMANNO TEXT PRIMARY KEY,
    SALESMANNAME TEXT,
    ADDRESS1 TEXT,
    ADDRESS2 TEXT,
    CITY TEXT,
    PINCODE INTEGER,
    STATE TEXT,
    SALAMT REAL,
    TGTTOGET REAL,
    YTDSALES REAL,
    REMARKS TEXT
);

CREATE TABLE SALES_ORDER (
    ORDERNO TEXT PRIMARY KEY,
    CLIENTNO TEXT,
    SALESMANNO TEXT,
    ORDERDATE TEXT,
    DELYADDR TEXT,
    DELYTYPE TEXT,
    BILLYN TEXT,
    DELYDATE TEXT,
    ORDERSTATUS TEXT,
    FOREIGN KEY (CLIENTNO) REFERENCES CLIENT_MASTER(CLIENTNO),
    FOREIGN KEY (SALESMANNO) REFERENCES SALESMAN_MASTER(SALESMANNO)
);

CREATE TABLE SALES_ORDER_DETAILS (
    ORDERNO TEXT,
    PRODUCTNO TEXT,
    QTYORDERED INTEGER,
    QTYDISP INTEGER,
    PRODUCTRATE REAL,
    PRIMARY KEY (ORDERNO, PRODUCTNO),
    FOREIGN KEY (ORDERNO) REFERENCES SALES_ORDER(ORDERNO),
    FOREIGN KEY (PRODUCTNO) REFERENCES PRODUCT_MASTER(PRODUCTNO)
);
""")
conn.commit()


## 3. Insert Sample Data

In [ ]:

clients = [
    ('C101','Aman','Mumbai',400001,'Maharashtra',12000),
    ('C102','Ravi','Bangalore',560001,'Karnataka',8000),
    ('C103','Karan','Mangalore',575001,'Karnataka',15000)
]

products = [
    ('P01','Laptop',10.0,'Nos',50,10,50000,45000),
    ('P02','Mouse',15.0,'Nos',200,20,500,300),
    ('P03','Keyboard',20.0,'Nos',150,15,800,600)
]

salesmen = [
    ('S01','Raj','Address1','Address2','Mumbai',400001,'Maharashtra',10000,50000,20000,'Good'),
    ('S02','Vijay','Address3','Address4','Bangalore',560001,'Karnataka',12000,60000,25000,'Excellent')
]

orders = [
    ('O01','C101','S01','2024-06-10','Mumbai','F','N','2024-06-15','Fulfilled'),
    ('O02','C102','S02','2024-07-12','Bangalore','F','N','2024-07-18','Fulfilled')
]

details = [
    ('O01','P01',1,1,50000),
    ('O01','P02',2,2,1000),
    ('O02','P03',3,3,2400)
]

cur.executemany("INSERT INTO CLIENT_MASTER VALUES (?,?,?,?,?,?)", clients)
cur.executemany("INSERT INTO PRODUCT_MASTER VALUES (?,?,?,?,?,?,?,?)", products)
cur.executemany("INSERT INTO SALESMAN_MASTER VALUES (?,?,?,?,?,?,?,?,?,?,?)", salesmen)
cur.executemany("INSERT INTO SALES_ORDER VALUES (?,?,?,?,?,?,?,?,?)", orders)
cur.executemany("INSERT INTO SALES_ORDER_DETAILS VALUES (?,?,?,?,?)", details)

conn.commit()


## 4. Queries

In [ ]:

# Clients whose second letter is 'a'
cur.execute("SELECT * FROM CLIENT_MASTER WHERE NAME LIKE '_a%'")
print(cur.fetchall())


In [ ]:

# Clients from Bangalore or Mangalore
cur.execute("SELECT * FROM CLIENT_MASTER WHERE CITY IN ('Bangalore','Mangalore')")
print(cur.fetchall())


In [ ]:

# Orders placed in June
cur.execute("SELECT * FROM SALES_ORDER WHERE strftime('%m', ORDERDATE)='06'")
print(cur.fetchall())


In [ ]:

# Product-wise total quantity sold
cur.execute("""
SELECT P.DESCRIPTION, SUM(D.QTYORDERED)
FROM PRODUCT_MASTER P
JOIN SALES_ORDER_DETAILS D ON P.PRODUCTNO=D.PRODUCTNO
GROUP BY P.DESCRIPTION
""")
print(cur.fetchall())

# 3b. Clients who stay in a city whose first letter is 'M'
print("\nClients from City starting with M:")
cur.execute("SELECT * FROM CLIENT_MASTER WHERE CITY LIKE 'M%'")
print(cur.fetchall())

# 3d. Clients whose BalDue is greater than 10000
print("\nClients with BalDue > 10000:")
cur.execute("SELECT * FROM CLIENT_MASTER WHERE BALDUE > 10000")
print(cur.fetchall())

# 3f. Order No & day on which clients placed their order
print("\nOrder No and Day:")
cur.execute("SELECT ORDERNO, strftime('%d', ORDERDATE) FROM SALES_ORDER")
print(cur.fetchall())

# 3g. Clients not in state of 'Maharashtra'
print("\nClients not in Maharashtra:")
cur.execute("SELECT NAME, CITY, STATE FROM CLIENT_MASTER WHERE STATE != 'Maharashtra'")
print(cur.fetchall())

# 4b. Avg quantity sold for each client that has a maximum order value of 15000.00
print("\nAvg Qty sold for High Value Clients:")
cur.execute("""
SELECT C.NAME, AVG(D.QTYORDERED)
FROM CLIENT_MASTER C
JOIN SALES_ORDER O ON C.CLIENTNO = O.CLIENTNO
JOIN SALES_ORDER_DETAILS D ON O.ORDERNO = D.ORDERNO
WHERE C.CLIENTNO IN (
    SELECT O2.CLIENTNO
    FROM SALES_ORDER O2
    JOIN SALES_ORDER_DETAILS D2 ON O2.ORDERNO = D2.ORDERNO
    GROUP BY O2.ORDERNO
    HAVING SUM(D2.QTYORDERED * D2.PRODUCTRATE) >= 15000
)
GROUP BY C.NAME
""")
print(cur.fetchall())


## 5. Close Connection

In [ ]:

conn.close()
